In [2]:
import numpy as np
import pandas as pd
import pickle
import xgboost

from catboost import CatBoostRegressor
from math import sqrt
from matplotlib import pyplot
from pymlx import *
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from xgboost import XGBRegressor

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
# load saved model
xgb_model = pickle.load(open('../data/results_data/models/xgBoost.bin', 'rb'))

# load predictions
new_data = pd.read_csv('../data/data_for_XGB-prediction.csv')

# choose the specific columns to keep from the predictions data
keep_cols = ['ndre_val', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
             'irradiance.w_per_m.2', 'avg_clay', 'avg_sand', 'avg_silt']
test_X = new_data[keep_cols]

# assign values as training data
X_test = test_X.values

# preview values
['ndre_val', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s',
 'irradiance.w_per_m.2', 'avg_clay', 'avg_sand', 'avg_silt']

,ndre_val,precip.cm,tair.C,rh.pct,wind_sp.m_per_s,irradiance.w_per_m.2,avg_clay,avg_sand,avg_silt
0,0.527599,0.0,16.823333,63.1625,0.466667,337.595833,34.544097,9.273535,56.182369
1,0.513371,0.0,16.823333,63.1625,0.466667,337.595833,34.544097,9.273535,56.182369
2,0.502703,0.0,16.823333,63.1625,0.466667,337.595833,34.544097,9.273535,56.182369
3,0.472289,0.0,16.823333,63.1625,0.466667,337.595833,34.544097,9.273535,56.182369
4,0.476290,0.0,16.823333,63.1625,0.466667,337.595833,34.544097,9.273535,56.182369


In [33]:
# calculate mean and standard deviation
mean_X = X_test.mean(axis=0)
std_X = X_test.std(axis=0)

# center and standardize data
X_test = (X_test - mean_X) / std_X
X_test = X_test.astype('float32')

# set values to known mean and standard deviation
mean_y = 0.22123822444691976
std_y = 0.05354998924136244

# calculate predictions
predictions = xgb_model.predict(X_test)
predictions = predictions*std_y+mean_y

# export predictions to csv
predictions = predictions.tolist()
predictions = pd.DataFrame({'predictions': predictions})
predictions.to_csv('../data/predictions.csv')